In [1]:
import warnings
warnings.filterwarnings("ignore")

import soundfile
from datasets import load_dataset
import os
from tqdm import tqdm
import glob

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

import numpy as np
from collections import Counter

ds = load_dataset("MushanW/GLOBE", cache_dir="/mntcephfs/data/audiow/shoinoue/Dataset/GLOBE/hf_hub/")

def preprocess_accent_label(a):
    a = a.lower()
    candidates = ["england", "liverpool", "british", "scottish", "irish", "welsh"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "british"
    if "german" in a:
        a = "german"
        
    if "english" in a:
        a = " ".join(a.split("english")[0].split())
        
    candidates = ["indi", "bangla"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "hindi"
        
    candidates = ["american", "united states", "texas", " us", "california", "drawl"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "american"
        
    candidates = ["canadian"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "canadian"
        
    candidates = ["african", "nigeria", "kenyan", "nepali", "afrika"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "african"
        
    candidates = ["russian", "ukrainian"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "russian"
        
    candidates = ["european", "transatlantic", "slavic", "austrian", "dutch", "polish", "german", "french", "bulgarian", "hunglish", "italian", "spanish", "swedish", "mexican", "georgian", "brazil", "swiss", "latin"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "european"
        
    candidates = ["malaysian", "hong kong", "japanese", "thai", "singaporean", "indonesian", "filipino", "cantonese"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "asian"
        
    candidates = ["australian", "new zealand", "kiwi"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "australian"
        
    candidates = ["transnational", "israeli", "russian", "african"]
    if np.sum(np.array([key in a for key in candidates]))>=1:
        a = "others"
    return a

Using the latest cached version of the dataset since MushanW/GLOBE couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /mntcephfs/data/audiow/shoinoue/Dataset/GLOBE/hf_hub/MushanW___globe/default/0.0.0/d39669170265a41c3dffa98e6d0cc027260def43 (last modified on Mon Jul 15 20:24:29 2024).


In [2]:
base_dir = "/mntcephfs/data/audiow/shoinoue/Dataset/GLOBE/rawfiles/"

# Save audiofiles individually

In [18]:
save_audio = False

# for mode in ["val", "test", "train"]:
for mode in ["train"]:
    print(mode)
    mode_dir = base_dir + f"{mode}/"
    os.makedirs(mode_dir, exist_ok=True)
    test = ds[mode]["audio"]
    speakerids = ds[mode]["speaker_id"]
    transcripts = ds[mode]["transcript"]
    accents = ds[mode]["accent"]
    genders = ds[mode]["gender"]
    ages = ds[mode]["age"]
    durations = ds[mode]["duration"]

    info = {} 
    for idx in tqdm(range(len(speakerids))):
        if save_audio:
            array = test[idx]["array"]
            sr = test[idx]["sampling_rate"]
        spkid = speakerids[idx]
        spk_dir = f"{mode_dir}{spkid}/"
        os.makedirs(spk_dir, exist_ok=True)
        savefile = f"{spk_dir}{spkid}_{test[idx]['path'][:-5]}.wav"

        if save_audio:
            soundfile.write(savefile, array, sr)

        info[os.path.basename(savefile)[:-4]] = {
            "speaker": spkid,
            "accent": accents[idx],
            "normalized accent": preprocess_accent_label(accents[idx]),
            "gender": genders[idx],
            "transcript": transcripts[idx],
            "age": ages[idx],
            "duration": durations[idx],
        }
    np.save(mode_dir+"information.npy", info)

train


100%|██████████| 572159/572159 [00:53<00:00, 10760.33it/s]


In [14]:
np.load(mode_dir+"information.npy", allow_pickle=True).item()

{'S_000658_000030': {'speaker': 'S_000658',
  'accent': 'australian',
  'gender': 'female',
  'transcript': 'each member has one share and one vote.'},
 'S_000658_000021': {'speaker': 'S_000658',
  'accent': 'australian',
  'gender': 'female',
  'transcript': 'true to his cynical nature, he continues to believe that life never gets better.'},
 'S_000658_000015': {'speaker': 'S_000658',
  'accent': 'australian',
  'gender': 'female',
  'transcript': 'the kurdistan region has an increasing urban population with still a significant rural population.'},
 'S_000658_000026': {'speaker': 'S_000658',
  'accent': 'australian',
  'gender': 'female',
  'transcript': 'the yeats sisters lived together through their adult lives, albeit contentiously.'},
 'S_000658_000031': {'speaker': 'S_000658',
  'accent': 'australian',
  'gender': 'female',
  'transcript': 'no official policy change has been mentioned regarding this.'},
 'S_000658_000027': {'speaker': 'S_000658',
  'accent': 'australian',
  'gend